In [1]:
import pandas as pd
import re
import joblib

class CourseRecommender:
    def __init__(self, courses_df):
        self.courses_df = courses_df
        self.courses_df['Skills'] = self.courses_df['Skills'].fillna('')  # Replace NaN with empty string

    def recommend_courses(self, field_of_interest, skills, enrolled_courses=None, num_recommendations=5):
        # Combine user interests and skills into a single string for easier matching
        interest_pattern = field_of_interest.strip()

        # Escape special characters in each skill and join them with '|'
        skills_pattern = '|'.join([re.escape(skill.strip()) for skill in skills])

        # Filter courses based on the field of interest (Category or Sub-Category)
        matching_courses = self.courses_df[
            (self.courses_df['Category'].str.contains(interest_pattern, case=False, na=False)) |
            (self.courses_df['Sub-Category'].str.contains(interest_pattern, case=False, na=False)) |
            (self.courses_df['Skills'].str.contains(skills_pattern, case=False, na=False))
        ]

        # If the user has enrolled in courses, exclude them from recommendations
        if enrolled_courses:
            enrolled_courses_cleaned = [course.strip() for course in enrolled_courses]
            matching_courses = matching_courses[~matching_courses['Title'].isin(enrolled_courses_cleaned)]

        # Select the top courses and return the title and URL
        recommendations = matching_courses[['Title', 'URL']].head(num_recommendations)

        return recommendations

In [2]:
# Load the data
courses_df = pd.read_csv(r'F:\\brightfuture\\ml_models\\course_recommendation\\online_courses_data (Final).csv')

# Create an instance of the recommender
recommender = CourseRecommender(courses_df)

# Save the recommender model
joblib.dump(CourseRecommender, 'course_recommender.joblib')

['course_recommender.joblib']

In [3]:
# Load the recommender model
loaded_recommender = joblib.load('course_recommender.joblib')

# Example usage:
field_of_interest = input("Enter your field of interest (e.g., Data Science, Business, AI): ")
skills_input = input("Enter your skills (comma-separated, e.g., Python, Machine Learning): ")
skills = skills_input.split(',')
enrolled_courses_input = input("Enter currently enrolled courses (comma-separated, or leave blank if none): ")
enrolled_courses = enrolled_courses_input.split(',') if enrolled_courses_input else None

# Call the recommendation function
recommended_courses = loaded_recommender.recommend_courses(
    field_of_interest,
    skills,
    enrolled_courses if enrolled_courses else None
)

# Display the recommendations
if not recommended_courses.empty:
    print("\nRecommended Courses:")
    for _, row in recommended_courses.iterrows():
        print(f"{row['Title']}")
        print(f"Link: {row['URL']}\n")
else:
    print("No courses found matching your input.")


Recommended Courses:
Machine Learning Specialization
Link: https://www.coursera.org/specializations/machine-learning-introduction

Introduction to Data Science Specialization
Link: https://www.coursera.org/specializations/introduction-data-science

Data Science Fundamentals with Python and SQL Specialization
Link: https://www.coursera.org/specializations/data-science-fundamentals-python-sql

Key Technologies for Business Specialization
Link: https://www.coursera.org/specializations/key-technologies-for-business

Deep Learning Specialization
Link: https://www.coursera.org/specializations/deep-learning

